In [9]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='forest-loss-monitoring')

# Task
Generate Python code to visualize the most recent Sentinel-1 VH and VV polarization data for Nuremberg Castle, Germany, step-by-step and concisely.

## Define the area of interest (aoi)

### Subtask:
Create a polygon or bounding box around Nuremberg Castle.


**Reasoning**:
Import the necessary library and define the area of interest around Nuremberg Castle using a bounding box.



In [10]:
from shapely.geometry import box

# Approximate coordinates for Nuremberg Castle
#latitude = 49.4542
#longitude = 11.0784

# Iceland location (66°08'21.8"N 15°29'45.7"W)
latitude = 66.139389
longitude = -15.496028

# Define a bounding box around the castle
# Using a small buffer around the coordinates
buffer = 0.1  # degrees

min_lon = longitude - buffer
min_lat = latitude - buffer
max_lon = longitude + buffer
max_lat = latitude + buffer

aoi = box(min_lon, min_lat, max_lon, max_lat)

## Search for sentinel-1 data

### Subtask:
Use the AOI and date range to find the most recent Sentinel-1 images with VH and VV polarization.


**Reasoning**:
Import necessary libraries and define the time range, instance ID, and create the SentinelHubRequest object including data collection, download task, evalscript task, and maxcc parameter.



### Search for Sentinel-1 data

**Reasoning**:
Filter the Sentinel-1 collection by date, area of interest, and polarization. Then, select the most recent image.

In [11]:
# Define the date range for the most recent data
from datetime import date, timedelta

end_date = date.today().strftime('%Y-%m-%d')
start_date = (date.today() - timedelta(days=30)).strftime('%Y-%m-%d') # Search in the last 30 days

# Convert the shapely AOI to Earth Engine geometry
ee_aoi = ee.Geometry.Polygon(list(aoi.exterior.coords))

# Load the Sentinel-1 collection
sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD')

# Filter the collection
filtered_collection = sentinel1.filterDate(start_date, end_date) \
                               .filterBounds(ee_aoi) \
                               .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
                               .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
                               .filter(ee.Filter.eq('instrumentMode', 'IW')) # Interferometric Wide Swath

# Select the most recent image
# most_recent_image = filtered_collection.sort('system:time_start', False).first()

# Mosaic the filtered collection to stitch images together
stitched_image = filtered_collection.mosaic()


# Display the most recent image information
# print("Most recent image acquisition date:", most_recent_image.get('system:time_start').getInfo())

In [16]:
import ee
import folium
import geemap
import numpy as np
from datetime import datetime, timedelta

In [18]:
# Select the VH and VV bands
vh_band = stitched_image.select('VH')
vv_band = stitched_image.select('VV')

# Define visualization parameters
vh_vis_params = {'min': -25, 'max': 0, 'palette': ['black', 'white']}
vv_vis_params = {'min': -25, 'max': 0, 'palette': ['black', 'white']}

# Create a Folium map
center_lat = (min_lat + max_lat) / 2
center_lon = (min_lon + max_lon) / 2
my_map = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Add EE layers using getMapId
vh_map_id = vh_band.getMapId(vh_vis_params)
vv_map_id = vv_band.getMapId(vv_vis_params)

folium.raster_layers.TileLayer(
    tiles=vh_map_id['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    name='Sentinel-1 VH',
    overlay=True,
    control=True
).add_to(my_map)

folium.raster_layers.TileLayer(
    tiles=vv_map_id['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    name='Sentinel-1 VV',
    overlay=True,
    control=True
).add_to(my_map)

# Add layer control
folium.LayerControl().add_to(my_map)

# Display the map
my_map

In [17]:
# Select the VH and VV bands
vh_band = stitched_image.select('VH')
vv_band = stitched_image.select('VV')

# Define visualization parameters for the bands
vh_vis_params = {'min': -25, 'max': 0}
vv_vis_params = {'min': -25, 'max': 0}

# Create a Folium map centered on the AOI
center_lat = (min_lat + max_lat) / 2
center_lon = (min_lon + max_lon) / 2
my_map = folium.Map(location=[center_lat, center_lon], zoom_start=12) # Reduced zoom for wider view

# Add the VH and VV layers to the map
my_map.add_ee_layer(vh_band, vh_vis_params, 'Sentinel-1 VH')
my_map.add_ee_layer(vv_band, vv_vis_params, 'Sentinel-1 VV')

# Add a layer control panel to the map
my_map.add_child(folium.LayerControl())

# Display the map
display(my_map)

AttributeError: 'Map' object has no attribute 'add_ee_layer'

In [20]:
# Load the Google Satellite Embedding Collection
embedding_collection = ee.ImageCollection('GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL')

# Filter the collection by the area of interest
filtered_embedding_collection = embedding_collection.filterBounds(ee_aoi)

# Select the most recent image
most_recent_embedding_image = filtered_embedding_collection.sort('system:time_start', False).first()

# Print information about the selected image
print("Most recent embedding image information:", most_recent_embedding_image.getInfo())

# Define visualization parameters
embedding_vis_params = {'bands': ['A00', 'A01', 'A02'], 'min': -0.6, 'max': 0.3}

# Create a Folium map centered on the AOI
center_lat = (min_lat + max_lat) / 2
center_lon = (min_lon + max_lon) / 2
embedding_map = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Add the embedding layer using getMapId (FIXED)
embedding_map_id = most_recent_embedding_image.getMapId(embedding_vis_params)

folium.raster_layers.TileLayer(
    tiles=embedding_map_id['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    name='Satellite Embedding',
    overlay=True,
    control=True
).add_to(embedding_map)

# Add layer control
folium.LayerControl().add_to(embedding_map)

# Display the map
embedding_map

Most recent embedding image information: {'type': 'Image', 'bands': [{'id': 'A00', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [16384, 16384], 'crs': 'EPSG:32628', 'crs_transform': [10, 0, 336160, 0, 10, 7208960]}, {'id': 'A01', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [16384, 16384], 'crs': 'EPSG:32628', 'crs_transform': [10, 0, 336160, 0, 10, 7208960]}, {'id': 'A02', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [16384, 16384], 'crs': 'EPSG:32628', 'crs_transform': [10, 0, 336160, 0, 10, 7208960]}, {'id': 'A03', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [16384, 16384], 'crs': 'EPSG:32628', 'crs_transform': [10, 0, 336160, 0, 10, 7208960]}, {'id': 'A04', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [16384, 16384], 'crs': 'EPSG:32628', 'crs_transform': [10, 0, 336160, 0, 10, 7208960]}, {'id': 'A05', 'data_type': {'type': 'PixelType', 'precis

## Access and Display Google Satellite Embedding Data

### Subtask:
Load an image from the Google Satellite Embedding Collection for the area of interest.

**Reasoning**:
Load the Google Satellite Embedding Collection, filter it by the area of interest and date, and select an image to display.

In [19]:
# Load the Google Satellite Embedding Collection
embedding_collection = ee.ImageCollection('GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL')

# Filter the collection by the area of interest
filtered_embedding_collection = embedding_collection.filterBounds(ee_aoi)

# Select the most recent image in the filtered collection (assuming there is one)
# You might need to adjust date filtering based on the dataset's available dates
most_recent_embedding_image = filtered_embedding_collection.sort('system:time_start', False).first()

# Print information about the selected image
print("Most recent embedding image information:", most_recent_embedding_image.getInfo())

# Define visualization parameters for the embedding image (this will depend on what you want to visualize)
# For demonstration, let's visualize the first three bands as RGB
# Updated min/max based on the calculated values for A00, A01, A02
embedding_vis_params = {'bands': ['A00', 'A01', 'A02'], 'min': -0.6, 'max': 0.3}


# Create a Folium map centered on the AOI
center_lat = (min_lat + max_lat) / 2
center_lon = (min_lon + max_lon) / 2
embedding_map = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Add the embedding layer to the map
embedding_map.add_ee_layer(most_recent_embedding_image, embedding_vis_params, 'Satellite Embedding')

# Add a layer control panel to the map
embedding_map.add_child(folium.LayerControl())

# Display the map
display(embedding_map)

Most recent embedding image information: {'type': 'Image', 'bands': [{'id': 'A00', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [16384, 16384], 'crs': 'EPSG:32628', 'crs_transform': [10, 0, 336160, 0, 10, 7208960]}, {'id': 'A01', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [16384, 16384], 'crs': 'EPSG:32628', 'crs_transform': [10, 0, 336160, 0, 10, 7208960]}, {'id': 'A02', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [16384, 16384], 'crs': 'EPSG:32628', 'crs_transform': [10, 0, 336160, 0, 10, 7208960]}, {'id': 'A03', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [16384, 16384], 'crs': 'EPSG:32628', 'crs_transform': [10, 0, 336160, 0, 10, 7208960]}, {'id': 'A04', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [16384, 16384], 'crs': 'EPSG:32628', 'crs_transform': [10, 0, 336160, 0, 10, 7208960]}, {'id': 'A05', 'data_type': {'type': 'PixelType', 'precis

AttributeError: 'Map' object has no attribute 'add_ee_layer'

### Inspect Band Values for Visualization

**Reasoning**:
Calculate and print the minimum and maximum values for the 'A00', 'A01', and 'A02' bands within the area of interest to determine appropriate visualization parameters.